In [0]:
import torch
import numpy as np

In [0]:
# Input (temp,rainfall, humidity)
inputs = np.array([[73,67,44],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]], dtype='float32')

In [0]:
#Targets (apples,oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]],dtype='float32')

In [0]:
# Convert Numpy arrays to Pytorch Tensors

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Linear Regression from Scratch

In [0]:
# Seeting Random Weights and Biases
# In Shape dim0 is 2 because we are fitting two linear regression models one for oranges and other for apples  
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2,requires_grad=True)

In [6]:
w,b

(tensor([[ 0.2545,  0.8543,  0.3456],
         [-1.2694, -1.3081, -1.1873]], requires_grad=True),
 tensor([ 0.4811, -0.3114], requires_grad=True))

### Model Definition

In [0]:
def model(x):
  return x @ w.t() + b

In [8]:
model(inputs) #Predictions

tensor([[  91.5078, -232.8610],
        [ 120.9423, -306.9261],
        [ 157.1496, -354.8994],
        [  75.9656, -229.9664],
        [ 124.2513, -296.5887]], grad_fn=<AddBackward0>)

In [9]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

### Loss Function

In [0]:
def mse(t1,t2):
  diff = t1-t2
  return torch.sum(diff*diff)/diff.numel()

In [11]:
loss = mse(model(inputs),targets)
loss

tensor(74783.4844, grad_fn=<DivBackward0>)

### Compute Gradients

In [0]:
loss.backward()

In [13]:
print(w)
print(w.grad)

tensor([[ 0.2545,  0.8543,  0.3456],
        [-1.2694, -1.3081, -1.1873]], requires_grad=True)
tensor([[  3303.3359,   3073.3306,   1963.1309],
        [-31516.7129, -34588.7539, -21340.0547]])


In [14]:
print(b)
print(b.grad)

tensor([ 0.4811, -0.3114], requires_grad=True)
tensor([  37.7634, -376.2483])


In [15]:
#Resetting Gradients
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

### Optimization

In [16]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(74783.4844, grad_fn=<DivBackward0>)


In [0]:
# Compute Gradients
loss.backward()

In [0]:
# Adjust weights and reset gradients
# We use __torch.no_grad()__ to indicate Pytorch that we shouldn't track, calculate or modify gradients while updating the gradinets and biases
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [0]:
preds = model(inputs)
loss = mse(preds,targets)

In [20]:
print(loss)

tensor(50481.5625, grad_fn=<DivBackward0>)


### Train for multiple epoches

In [0]:
for i in range(400):
  preds = model(inputs)
  loss = mse(preds,targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [0]:
preds = model(inputs)

In [23]:
preds

tensor([[ 58.1572,  71.5566],
        [ 80.1908,  96.6024],
        [122.0009, 140.4593],
        [ 24.0916,  42.4298],
        [ 96.7738, 108.8623]], grad_fn=<AddBackward0>)

In [24]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [25]:
mse(targets,preds)

tensor(26.7113, grad_fn=<DivBackward0>)

# Liner Regression using Pytorch Builtins

In [0]:
import torch.nn as nn

In [0]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

### DataSet and DataLoader

In [0]:
from torch.utils.data import TensorDataset

In [29]:
#Define Dataset as Tuples of Inputs and Outputs 
train_ds = TensorDataset(inputs,targets)
train_ds

In [30]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [0]:
from torch.utils.data import DataLoader

In [32]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)
train_dl

In [33]:
for xb,yb in train_dl:
  print('batch:')
  print(xb)
  print(yb)

batch:
tensor([[ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [102.,  43.,  37.]])
tensor([[103., 119.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 22.,  37.]])
batch:
tensor([[ 87., 134.,  58.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [119., 133.],
        [ 56.,  70.],
        [119., 133.],
        [ 56.,  70.]])
batch:
tensor([[91., 88., 64.],
        [91., 88., 64.],
        [91., 88., 64.],
        [69., 96., 70.],
        [69., 96., 70.]])
tensor([[ 81., 101.],
        [ 81., 101.],
        [ 81., 101.],
        [103., 119.],
        [103., 119.]])


### nn.Linear

In [34]:
#Define Model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1410, -0.2952, -0.1258],
        [ 0.3888, -0.2265, -0.4987]], requires_grad=True)
Parameter containing:
tensor([0.0548, 0.2654], requires_grad=True)


In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1410, -0.2952, -0.1258],
         [ 0.3888, -0.2265, -0.4987]], requires_grad=True),
 Parameter containing:
 tensor([0.0548, 0.2654], requires_grad=True)]

In [36]:
preds = model(inputs)
preds

tensor([[-14.8406,  -7.9740],
        [-21.1437, -16.2057],
        [-34.5302, -25.1888],
        [ -2.9139,  11.7304],
        [-27.3611, -29.5640],
        [-14.8406,  -7.9740],
        [-21.1437, -16.2057],
        [-34.5302, -25.1888],
        [ -2.9139,  11.7304],
        [-27.3611, -29.5640],
        [-14.8406,  -7.9740],
        [-21.1437, -16.2057],
        [-34.5302, -25.1888],
        [ -2.9139,  11.7304],
        [-27.3611, -29.5640]], grad_fn=<AddmmBackward>)

### Loss Function

In [0]:
import torch.nn.functional as F

In [0]:
loss_fn = F.mse_loss

In [39]:
loss = loss_fn(preds,targets)
loss

tensor(12418.8633, grad_fn=<MseLossBackward>)

### Optimizer

In [0]:
opt = torch.optim.SGD(model.parameters(),lr=1e-5)
#model.parameters() is passed to the optimizer, so that the optimizer knows which matrices should be modified during update step

In [0]:
?torch.optim.SGD

### Train The Model

In [0]:
def fit(num_epoches,model,loss_fn,optimizer):
  for epoch in range(num_epoches):
    for xb,yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred,yb)
      loss.backward()
      optimizer.step()#update the parameters using gradients
      optimizer.zero_grad()#Reset the Gradients to zero
    if (epoch+1)%10 == 0:
      print('Epoch [{}/{}], loss={:.4f}'.format(epoch+1,num_epoches,loss.item()))

In [44]:
fit(200,model,loss_fn,opt)

Epoch [10/200], loss=37.6933
Epoch [20/200], loss=33.1223
Epoch [30/200], loss=42.1775
Epoch [40/200], loss=18.2132
Epoch [50/200], loss=36.6152
Epoch [60/200], loss=24.8020
Epoch [70/200], loss=34.6657
Epoch [80/200], loss=40.7081
Epoch [90/200], loss=17.5634
Epoch [100/200], loss=12.8975
Epoch [110/200], loss=14.6100
Epoch [120/200], loss=25.9579
Epoch [130/200], loss=23.9468
Epoch [140/200], loss=10.2014
Epoch [150/200], loss=17.0110
Epoch [160/200], loss=12.2027
Epoch [170/200], loss=19.7517
Epoch [180/200], loss=13.4724
Epoch [190/200], loss=16.8455
Epoch [200/200], loss=8.4761


In [0]:
model(inputs)

In [0]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])